#  New York City Taxi and Limousine Commission Project

---

**This notebook has a goal apply de PACE strategy on 2017_Yellow_Taxi_Trip_Data dataset to build a predictive model for total amount of a taxi trip**

---

## **PACE: Plan**

This stage is where i've the first contact with the dataset and when i start to plan what to do with the data to achieve my goals. Normally, I start making some questions to dataset. Let's get start  (Remenber, this first questions don't have high accuracy )

### What are the data columns most relevant to your deliverable?

  - tpep_pickup_datetime, tpep_dropoff_datetime: To calculate the trip duration 
  - Trip_distance: The elapsed trip distance in miles reported by the taximeter.
  - Fare_amount: The time-and-distance fare calculated by the meter.
  - Tip_amount: Tip amount – This field is automatically populated for credit card tips. Cash tips are not included.
  - Tolls_amount: Total amount of all tolls paid in trip. 
  - Total_amount: The total amount charged to passengers. Does not include cash tips.

### What are the data columns most irrelevant to your deliverable?

  - Store_and_fwd_flag: dont have correlation in all dataset  


### Are in your dataset missing or incomplete data?
  No, but i have incositent data. The columns  with are: 
  - Fare_amount: values 0 or negative 
  - Extra: negative values
  - Mta_tax: negative values
  - Improvement_surcharge: values 0 or negative
  - Total_amount: values 0 or negative
  
### Which EDA practices will be required to begin this project?
  - Data cleaning & Validity checks
  - Temporal Analysis
  - Univariate Analysis (Distribution of Target)
  - Bivariate Analysis (Feature Relationships)
  

## **Pace: Analyze**

In this stage i start data cleaning and validade checks and witch steps is i've do to achive the best resoults. We can continue with some questions. 


### What steps need to be taken to perform EDA in the most effective way to achieve the project goal?
  #### First: I start plotting my data with Ydata to start my analisys (this is valid for Plan fase too)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

df_taxi = pd.read_csv("2017_Yellow_Taxi_Trip_Data.csv")
df_taxi.drop("ID", axis=1, inplace=True)
df_taxi.drop("VendorID", axis=1, inplace=True)
profile_taxi = ProfileReport(df_taxi, title="Taxi analysis with ydata", explorative=True)
profile_taxi.to_file("taxi_analysis_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:05<00:00,  2.77it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

#### Second: I start cleaning inconsistant data i find in my plan fase

In [2]:
df_taxi.drop(columns=["store_and_fwd_flag"], inplace=True)
df_taxi['tpep_pickup_datetime'] = pd.to_datetime(df_taxi['tpep_pickup_datetime'])
df_taxi['tpep_dropoff_datetime'] = pd.to_datetime(df_taxi['tpep_dropoff_datetime'])
df_taxi = df_taxi[df_taxi['fare_amount'] > 0]
df_taxi = df_taxi[df_taxi['extra'] > 0]
df_taxi = df_taxi[df_taxi['mta_tax'] > 0]
df_taxi = df_taxi[df_taxi['improvement_surcharge'] > 0]
df_taxi = df_taxi[df_taxi['total_amount'] > 0]


#### Third: Now i add a new columns to help analize data


In [7]:
df_taxi['duration'] = df_taxi['tpep_dropoff_datetime'] - df_taxi['tpep_pickup_datetime']
df_taxi['duration_minutes'] = df_taxi['duration'].dt.total_seconds() / 60
df_taxi.drop(columns=['duration'], inplace=True)

#### Then i get anoter report with ydata and reanalyze

In [8]:
profile_taxi = ProfileReport(df_taxi, title="Taxi analysis with ydata", explorative=True)
profile_taxi.to_file("taxi_analysis_report_cleaned.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:00<00:00, 214.46it/s]
c:\Users\smota\AppData\Local\Programs\Python\Python313\Lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with timedelta64[ns], please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]